In [1]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from astroquery.gaia import Gaia
#from scipy.optimize import least_squares
from scipy.optimize import curve_fit

Created TAP+ (v1.2.1) - Connection:
	Host: gea.esac.esa.int
	Use HTTPS: True
	Port: 443
	SSL Port: 443
Created TAP+ (v1.2.1) - Connection:
	Host: geadata.esac.esa.int
	Use HTTPS: True
	Port: 443
	SSL Port: 443


In [4]:
job = Gaia.launch_job_async("select top 10000 * from gaiadr2.vari_cepheid order by source_id")
r = job.get_results()

ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host

In [ ]:
r

In [ ]:
np.sort(r['pf'])

In [ ]:
sources = r['source_id']

### Light curves with flux

In [ ]:
lcs = np.loadtxt('light_curves_first.txt',skiprows=1,delimiter=',',usecols=(0,3,4,5,6,7))
lcs_band = np.loadtxt('light_curves_first.txt',skiprows=1,delimiter=',',usecols=2, dtype=np.str)

In [ ]:
source_id = lcs[:,0]
band = lcs_band
time = lcs[:,1]
mag = lcs[:,2]
flux = lcs[:,3]
flux_error = lcs[:,4] 
flux_over_error = lcs[:,5]
print(source_id)

In [ ]:
#First example
asas_ind = np.argwhere(source_id==2957940098405233024)
#asas_ind = np.argwhere(source_id==6909188166792514944)
asas_mag = mag[asas_ind[0][0]:asas_ind[-1][0]]
asas_time = time[asas_ind[0][0]:asas_ind[-1][0]]
asas_flux = flux[asas_ind[0][0]:asas_ind[-1][0]]
asas_flux_error = flux_error[asas_ind[0][0]:asas_ind[-1][0]]
asas_flux_over_error = flux_over_error[asas_ind[0][0]:asas_ind[-1][0]]
#Seperating bands
asas_band = band[asas_ind[0][0]:asas_ind[-1][0]]
G_ind = np.argwhere(asas_band=='G')
BP_ind = np.argwhere(asas_band=='BP')
RP_ind = np.argwhere(asas_band=='RP')
asas_time_G = asas_time[G_ind[0][0]:G_ind[-1][0]]
asas_time_BP = asas_time[BP_ind[0][0]:BP_ind[-1][0]]
asas_time_RP = asas_time[RP_ind[0][0]:RP_ind[-1][0]]
#time_BP and time_RP are nearly identical to time_G, but have different lengths
asas_mag_G = asas_mag[G_ind[0][0]:G_ind[-1][0]]
asas_mag_BP = asas_mag[BP_ind[0][0]:BP_ind[-1][0]]
asas_mag_RP = asas_mag[RP_ind[0][0]:RP_ind[-1][0]]
asas_flux_G = asas_flux[G_ind[0][0]:G_ind[-1][0]]
asas_flux_error_G = asas_flux_error[G_ind[0][0]:G_ind[-1][0]]
asas_flux_over_error_G = asas_flux_over_error[G_ind[0][0]:G_ind[-1][0]]
asas_flux_BP = asas_flux[BP_ind[0][0]:BP_ind[-1][0]]
asas_flux_RP = asas_flux[RP_ind[0][0]:RP_ind[-1][0]]
#asas_flux = flux[asas_ind[0][0]:asas_ind[-1][0]]

In [ ]:
plt.plot(asas_time_G, asas_mag_G, 'g.-')
plt.plot(asas_time_BP, asas_mag_BP, 'b.-')
plt.plot(asas_time_RP, asas_mag_RP, 'r.-')
plt.title('Light Curve')
plt.xlabel('Time')
plt.ylabel('Mag')
plt.gca().invert_yaxis()
plt.show()

plt.plot(asas_time_G, asas_flux_G, 'g.-')
plt.plot(asas_time_BP, asas_flux_BP, 'b.-')
plt.plot(asas_time_RP, asas_flux_RP, 'r.-')
plt.title('Light Curve')
plt.xlabel('Time')
plt.ylabel('Flux')
plt.show()

## Flux with error bars (better example below)

In [ ]:
print(asas_flux_error_G)

In [ ]:
#Magnitude of errors are, apparently, quite small.
#%matplotlib notebook
plt.errorbar(asas_time_G, asas_flux_G, yerr=asas_flux_error_G, ecolor='r')
plt.title('Light Curve')
plt.xlabel('Time')
plt.ylabel('Flux')
plt.show()

### Regarding the determination of periods:
https://gea.esac.esa.int/archive/documentation/GDR2/Gaia_archive/chap_datamodel/sec_dm_variability_tables/ssec_dm_vari_cepheid.html

"This value [pf] is obtained by modelling the G band time series using the Levenberg-Marquardt non linear fitting algorithm (see Clementini et al. 2016, A&A, 595, A133)."

Also worth noting, from the same webpage:

"The intensity-averaged magnitude is obtained by computing the average flux and then converting the average flux to magnitude."

## Plotting Phase

What the hell am I supposed to do when a source has a period on the order days, but the light curve only has points on the order minutes?
Nevermind, I now believe the units of the light curve time series is days.

In [ ]:
per = r['pf'][np.argwhere(r['source_id']==2957940098405233024)]
per = per[0][0]
#per = per*60*60*24 #Converting days to seconds
#Now I believe the time series is in days, as in Julian Days (minus refrence date))
phase = (asas_time_G % per)#/per
plt.scatter(phase, asas_flux_G, color='g')
plt.show()

In [ ]:
#Defining Source
#mysource = 2952257272558933248
#mysource = 2950975929193648128 #Maybe this source good?
#mysource = 2950541415941942016 #very bad
mysource = 2945995416399061760 #This source is decent!
#mysource = 2957940098405233024 #This is okayish.
asas_ind = np.argwhere(source_id==mysource)
asas_mag = mag[asas_ind[0][0]:asas_ind[-1][0]]
asas_time = time[asas_ind[0][0]:asas_ind[-1][0]]
asas_flux = flux[asas_ind[0][0]:asas_ind[-1][0]]
asas_band = band[asas_ind[0][0]:asas_ind[-1][0]]
G_ind = np.argwhere(asas_band=='G')
asas_time_G = asas_time[G_ind[0][0]:G_ind[-1][0]]
asas_mag_G = asas_mag[G_ind[0][0]:G_ind[-1][0]]
asas_flux_G = asas_flux[G_ind[0][0]:G_ind[-1][0]]
#Calculating phase
per = r['pf'][np.argwhere(r['source_id']==mysource)]
per = per[0][0]
print(per)
phase = (asas_time_G % per)#/per
plt.scatter(phase, asas_flux_G, color='g')
plt.title('Light Curve for ' + str(mysource) +', Per = '+ str('%.3f'%per))
plt.xlabel('Phase')
plt.ylabel('G Band Flux')
#plt.savefig('LCPhase'+str(mysource)+'.png', bboxes='tight', dpi=300)
plt.show()

In [ ]:
plt.scatter(phase, asas_mag_G, color='g')
plt.title('Light Curve for ' + str(mysource) +', Per = '+ str('%.3f'%per))
plt.xlabel('Phase')
plt.ylabel('Mag')
plt.gca().invert_yaxis()
#plt.savefig('LCPhase'+str(mysource)+'_Mag.png', bboxes='tight', dpi=300)
plt.show()

## Flux with error bars

In [ ]:
#Magnitude of errors are, apparently, quite small.
asas_flux_error = flux_error[asas_ind[0][0]:asas_ind[-1][0]]
asas_flux_error_G = asas_flux_error[G_ind[0][0]:G_ind[-1][0]]
plt.errorbar(phase, asas_flux_G, yerr=asas_flux_error_G, ls='None', ecolor='k', capsize=2)
plt.scatter(phase, asas_flux_G, color='g')
plt.title('Light Curve for ' + str(mysource) +', Per = '+ str('%.3f'%per))
plt.xlabel('Phase')
plt.ylabel('G Band Flux')
#plt.savefig('LCPhase'+str(mysource)+'.png', bboxes='tight', dpi=300)
plt.show()

## Raw Time Series Light Curve of My Source

In [ ]:
plt.errorbar(asas_time_G, asas_flux_G, yerr=asas_flux_error_G, ls='None', ecolor='k', capsize=2)
plt.plot(asas_time_G, asas_flux_G, 'g.-')
plt.title('Raw Light Curve for '+str(mysource))
plt.ylabel('G Band Flux (e-/s)')
plt.xlabel('Barycentric JD in TCB - 2455197.5 (days)')
#plt.savefig('LCRaw'+str(mysource)+'.png', bboxes='tight', dpi=300)
plt.show()

A rather ugly raw light curve. I think my mistake earlier today was trying to make nice looking phased light curves only out of raw light curves that looked nice.

## Mean of G Band Flux

In [ ]:
np.mean(asas_flux_G)
#len(asas_time_G) == 42

Above is merely the mean of the raw observed flux. What I want is the mean of a modeled flux.

## Modeling a Fourier Series to the Light Curve

In [ ]:
myperiod = r['pf'][np.argwhere(r['source_id']==mysource)]
myperiod = myperiod[0][0]
mytime = asas_time_G
myflux = asas_flux_G
mymag = asas_mag_G
def fourier3(tau,c,a,b):
    #argument: tau = (2*np.pi*t)/P 
    return c + a*np.sin(tau) + b*np.cos(tau)

def fit_fourier3(t, p, f):
    #c_min = 0
    #c_max = np.inf
    #bound_min = [c_min,c_min,c_min]
    #bound_max = [c_max,c_max,c_max]
    #bound = (bound_min,bound_max)
    popt = curve_fit(fourier3, t%p, f, bounds=(0,np.inf))#=bound)
    pcov = popt[1]
    popt = popt[0]
    #c = popt[0]
    #a = popt[1]
    #b = popt[2]
    return popt #c, a, b

In [ ]:
fit_fourier3(mytime, myperiod, myflux)

In [ ]:
fit_c = fit_fourier3(mytime, myperiod, myflux)[0]
fit_a = fit_fourier3(mytime, myperiod, myflux)[1]
fit_b = fit_fourier3(mytime, myperiod, myflux)[2]
tau = mytime #2*np.pi*mytime/myperiod
#tau = np.linspace(0,10,num=42)
fit = fit_c + fit_a*np.sin(tau) + fit_b*np.cos(tau)
#plt.plot(mytime, fit, c='r')
#plt.plot(mytime, myflux, c='g')
#plt.show()

In [ ]:
tau = np.linspace(0,2.6,num=42)#mytime #2*np.pi*mytime/myperiod
#tau = mytime % myperiod #The same, as I've defined it now (8am)
fit = fit_c + fit_a*np.sin(tau) + fit_b*np.cos(tau)
#plt.plot(tau, fit, c='r')
#plt.scatter(mytime % myperiod, myflux, c='g')
#plt.show()

### Fourier series fit

In [ ]:
N = 42 # number of data points
myt = mytime % myperiod
data = myflux
#first guesses
guess_freq = 1
guess_amplitude = 3*np.std(data)/(2**0.5)
guess_phase = 0
guess_offset = np.mean(data)
p0=[guess_freq, guess_amplitude, guess_phase, guess_offset]
#create the function we want to fit
def my_sin(x, freq, amplitude, phase, offset):
    return offset + (np.sin(x * freq + phase) * amplitude) + (np.cos(x * freq + phase) * amplitude)
#now do the fit
fit = curve_fit(my_sin, myt, data, p0=p0)
#recreate the fitted curve using the optimized parameters
data_fit = my_sin(myt, *fit[0])
plt.plot(data, '.', label='light curve points',c='g')
plt.plot(data_fit, label='light curve fit', c='r')
plt.legend()
plt.show()

In [ ]:
plt.plot(mytime, data, label='light curve points',c='g')
plt.plot(mytime, data_fit, label='light curve fit', c='r')
plt.legend()
plt.show()

In [ ]:
plt.scatter(mytime%myperiod, data, label='light curve points',c='g')
plt.scatter(mytime%myperiod, data_fit, label='light curve fit', c='r')
plt.legend()
plt.show()

In [ ]:
myfreq = fit[0][0] 
myamp = fit[0][1]  
myphase = fit[0][2] 
myoffset = fit[0][3]
x = np.linspace(0,2.6,num=3125)
fitcurve = myoffset + (np.sin(x*myfreq+myphase)*myamp) + (np.cos(x*myfreq+myphase)*myamp)
plt.scatter(mytime%myperiod, myflux, c='g')
plt.plot(x, fitcurve, c='r')
plt.show()

In [ ]:
x = np.linspace(min(mytime%myperiod),max(mytime%myperiod),num=3125)
fitcurve = myoffset + (np.sin(x*myfreq+myphase)*myamp) + (np.cos(x*myfreq+myphase)*myamp)
plt.errorbar(phase, asas_flux_G, yerr=asas_flux_error_G, ls='None', ecolor='k', capsize=2)
plt.scatter(mytime%myperiod, myflux, c='g')
plt.plot(x, fitcurve, c='r')
plt.title('Fitted Light Curve for ' + str(mysource))
plt.xlabel('Phase')
plt.ylabel('G Band Flux')
#plt.savefig('LCPhase'+str(mysource)+'_Fit.png', bboxes='tight', dpi=300)
plt.show()

In [ ]:
print(np.mean(data))
print(np.mean(data_fit))
print(np.mean(fitcurve))

## Same method as above, but with magnitude instead of flux

In [3]:
N = 42 # number of data points
myt = mytime % myperiod
t = np.linspace(0, 4*np.pi, N)
data = mymag

guess_freq = 1
guess_amplitude = 3*np.std(data)/(2**0.5)
guess_phase = 0
guess_offset = np.mean(data)

p0=[guess_freq, guess_amplitude,
    guess_phase, guess_offset]

# create the function we want to fit
def my_sin(x, freq, amplitude1, amplitude2, amplitude3, amplitude4, offset):
    deg1 = (np.sin(2*np.pi*x*freq)*amplitude1) + (np.cos(2*np.pi*x*freq)*amplitude2)
    deg2 = (np.sin(2*np.pi*2*x*freq)*amplitude3) + (np.cos(2*np.pi*2*x*freq)* amplitude4)
    return offset + deg1 + deg2

# now do the fit
fit_mag = curve_fit(my_sin, myt, data, p0=p0)

# recreate the fitted curve using the optimized parameters
data_fit_mag = my_sin(myt, *fit_mag[0])

plt.plot(mymag, '.', label='light curve points',c='g')
plt.plot(data_fit_mag, label='light curve fit', c='r')
plt.legend()
plt.show()

NameError: name 'mytime' is not defined

In [ ]:
plt.scatter(mytime%myperiod, mymag,c='g')
plt.gca().invert_yaxis()
plt.scatter(mytime%myperiod, data_fit_mag, label='light curve fit', c='r')
#plt.legend()
plt.show()

In [ ]:
magfreq = fit_mag[0][0] 
magamp = fit_mag[0][1]  
magphase = fit_mag[0][2] 
magoffset = fit_mag[0][3]
x = np.linspace(min(mytime%myperiod),max(mytime%myperiod),num=3125)
fitcurvemag = magoffset + (np.sin(x*magfreq+magphase)*magamp) + (np.cos(x*magfreq+magphase)*magamp)
plt.scatter(mytime%myperiod, mymag, c='g')
plt.plot(x, fitcurvemag, c='r')
plt.gca().invert_yaxis()
plt.title('Fitted Light Curve for ' + str(mysource))
plt.xlabel('Phase')
plt.ylabel('Mag')
#plt.savefig('LCPhase'+str(mysource)+'_Fit_Mag.png', bboxes='tight', dpi=300)
plt.show()

In [ ]:
print(fit_mag[0])
print(fit[0])

In [ ]:
print(np.mean(mymag))
print(np.mean(data_fit_mag))
print(np.mean(fitcurvemag))
print(r['int_average_g'][np.argwhere(r['source_id']==mysource)])

In [ ]:
fit[0]